# 개인 프로젝트
- 전국 뚜레쥬르 지점을 크롤링하여 지도에 나타내기

---
### 패키지 설치

In [ ]:
# BeautifulSoup
from bs4 import BeautifulSoup
import urllib.request as req

# SSL 인증서 검증 문제
## !pip install --pyOpenSSL
import ssl
import urllib.parse

# selenium(웹 브라우저를 자동화)
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

import numpy as np

---
### 크롤링 할 주소
- https://www.tlj.co.kr:7008/store/search.asp

In [ ]:
# Chrome Brower와 Crome Driver Version 확인하기
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.get('https://www.tlj.co.kr:7008/store/search.asp')

---
### 위치 정보 가져오기

In [ ]:
# 담을 리스트 선언
store, address, lat, lng, error_locations = [], [], [], [], []

> 패턴 상황      
첫 번째는 지점 이름       
두 번째는 도로명 주소 + 지번 주소

In [ ]:
# API로 위도, 경도값 호출 함수
def latlng(location):
    # API ID와 비밀번호 정의
    client_id = "dbb1i6yfa6"
    client_secret = "vL6zylRREWL0HA7SKDrW1l8IAuwbhajjvsplQL2w"

    # 찾고자 하는 주소 url 정의
    encText = urllib.parse.quote(location)
    url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=" + encText

    # 헤더에 ID와 비밀번호 추가
    requestUrl = urllib.request.Request(url)
    requestUrl.add_header("X-NCP-APIGW-API-KEY-ID", client_id)
    requestUrl.add_header("X-NCP-APIGW-API-KEY", client_secret)

    # 주소 불러오기
    response = urllib.request.urlopen(requestUrl)
    # 코드로 번역
    resStatusCode  = response.getcode()

    # 오류 잡기
    match resStatusCode :
        # 200은 성공했을 때 말함
        case 200 :
            response_body = response.read()
            entext = response_body.decode("utf-8")
        # 에러인 경우
        case _ :
            print("Error Code :", resStatusCode)

    # JSON으로 번역
    import json
    data1 = json.loads(entext)
    return data1

In [ ]:
# 4개 리스트 저장 함수
def saveList():

    # 위치 정보 전체 리스트 위치 변수 담기
    location_info_list_raw = driver.find_element(By.XPATH, '//*[@id="storeList"]')
    # 위치 정보 리스트 태그 변수 담기
    location_info_list = location_info_list_raw.find_elements(By.TAG_NAME, 'a')
    # 위치 정보 리스트 변수에 담기
    location_info = [option.get_attribute('text') for option in location_info_list]
    
    # error_locations = []  # 에러가 발생한 위치 정보를 저장할 리스트

    for index, data in enumerate(location_info):
        # 인덱스가 짝수일 때
        if index % 2 == 0:
            # 지점 이름 저장
            name = data.replace('뚜레쥬르 ','')+'점'
            store.append(name)
            # print(store)
        # 인덱스가 홀수일 때
        if index % 2 == 1:
            # 도로명 주소 저장
            # exc_location_01 = data.replace(' [오생성]','')
            # exc_location_02 = exc_location_01.replace(' (홍제동,홍제역)','')
            # exc_location_03 = exc_location_02.replace('내곡동 1111','헌릉로8길 58')
            # exc_location_04 = exc_location_03.replace('화중로104번 24 (송화빌딩) 1층 108호','화중로104번길 24 1층 108호')
            # exc_location_05 = exc_location_04.replace('대구달서구','대구광역시')
            # exc_location_06 = exc_location_05.replace('수색로20길 20-22 (수색동) DMC SK뷰아이파크포레 112호','수색로 322 상가 1층')
            location = data.replace('\t','').split('\n')[3].split(' 뚜레쥬르')[0]
            address.append(location)
            # print(address)
            # 성공했을 때
            try:
                # 위도와 경도 저장
                lng.append(latlng(location)['addresses'][0]['x'])
                lat.append(latlng(location)['addresses'][0]['y'])
                print(f"{name}의 정보 저장 - 현재 {len(address)}개 담김")
            # 에러일 때
            except Exception as e:
                print(f"에러에러!: {location} - {e}")
                lng.append(np.nan)
                lat.append(np.nan)
                error_locations.append((location, len(address)))
        

In [ ]:
count = 1
saveList()
print(f'---{count}페이지!---')
index = 3
while True:
    if 3 <= index <= 11:
        xpath = f'//*[@id="content"]/div/div/div[2]/div[1]/div[3]/span[{index}]/a'
        element_sel_gu = driver.find_element(By.XPATH, xpath)
        element_sel_gu.click()
        time.sleep(2)
        saveList()
        count += 1
        print(f'---{count}페이지!---')

    elif index == 12:
        index = 12
        xpath = f'//*[@id="content"]/div/div/div[2]/div[1]/div[3]/span[{index}]/a'
        element_sel_gu = driver.find_element(By.XPATH, xpath)
        element_sel_gu.click()
        time.sleep(2)
        index = 2
        xpath = f'//*[@id="content"]/div/div/div[2]/div[1]/div[3]/span[{index}]/a'
        element_sel_gu = driver.find_element(By.XPATH, xpath)
        element_sel_gu.click()
        time.sleep(2)
        saveList()
        count += 1
        print(f'---{count}페이지!---')
            
    index = index + 1 if index < 12 else 3
    
    # 필요한 경우 대기 시간 추가
    time.sleep(2)  # 1초 대기 (필요에 따라 조정)

In [1]:
import pandas as pd

In [ ]:
df = pd.DataFrame(
    {
        'store' : store,
        'address' : address,
        'lat' : lat,
        'lng' : lng,
    }
)
df

In [ ]:
df.to_csv('location.csv', index=False)

In [2]:
df = pd.read_csv('location.csv')
df

,store,address,lat,lng
0,순천법원점,전남 순천시 왕지5길 11,34.972200,127.523603
1,청주강서점,충북 청주시 흥덕구 서현중로35 1층,36.621703,127.429082
2,부산정관점,부산광역시 기장군 정관읍 정관로 395 (모전리) 102호,35.334160,129.166000
3,성가롤로병원점,전라남도 순천시 순광로 221,34.964733,127.543707
4,진영이진점,경상남도 김해시 진영읍 진산대로 63-11 (진영리) 1층,35.312084,128.726209
...,...,...,...,...
1336,경남거창점,경상남도 거창군 거창읍 아림로 31,35.685987,127.910475
1337,성남태평점,경기도 성남시 수정구 제일로 165-1 (태평동),37.441158,127.131438
1338,삼천포벌리점,경상남도 사천시 벌리로 2 (벌리동),34.938541,128.079910
1339,충북금왕점,충청북도 음성군 금왕읍 금석로 86,36.992922,127.597270


In [3]:
df['lat'].isna()
df1 = df[~df['lat'].isna()]
df1

,store,address,lat,lng
0,순천법원점,전남 순천시 왕지5길 11,34.972200,127.523603
1,청주강서점,충북 청주시 흥덕구 서현중로35 1층,36.621703,127.429082
2,부산정관점,부산광역시 기장군 정관읍 정관로 395 (모전리) 102호,35.334160,129.166000
3,성가롤로병원점,전라남도 순천시 순광로 221,34.964733,127.543707
4,진영이진점,경상남도 김해시 진영읍 진산대로 63-11 (진영리) 1층,35.312084,128.726209
...,...,...,...,...
1336,경남거창점,경상남도 거창군 거창읍 아림로 31,35.685987,127.910475
1337,성남태평점,경기도 성남시 수정구 제일로 165-1 (태평동),37.441158,127.131438
1338,삼천포벌리점,경상남도 사천시 벌리로 2 (벌리동),34.938541,128.079910
1339,충북금왕점,충청북도 음성군 금왕읍 금석로 86,36.992922,127.597270


In [4]:
df1.reset_index(inplace=True)

In [5]:
df2 = df1.drop('index', axis=1)
df2

,store,address,lat,lng
0,순천법원점,전남 순천시 왕지5길 11,34.972200,127.523603
1,청주강서점,충북 청주시 흥덕구 서현중로35 1층,36.621703,127.429082
2,부산정관점,부산광역시 기장군 정관읍 정관로 395 (모전리) 102호,35.334160,129.166000
3,성가롤로병원점,전라남도 순천시 순광로 221,34.964733,127.543707
4,진영이진점,경상남도 김해시 진영읍 진산대로 63-11 (진영리) 1층,35.312084,128.726209
...,...,...,...,...
1298,경남거창점,경상남도 거창군 거창읍 아림로 31,35.685987,127.910475
1299,성남태평점,경기도 성남시 수정구 제일로 165-1 (태평동),37.441158,127.131438
1300,삼천포벌리점,경상남도 사천시 벌리로 2 (벌리동),34.938541,128.079910
1301,충북금왕점,충청북도 음성군 금왕읍 금석로 86,36.992922,127.597270


In [6]:
import folium

In [7]:
# 서울 지도 만들기
korea_map = folium.Map(
    location=[37.55, 126.98],
    zoom_start=10
)

# 대학교 위치 정보를 Marker로 표시
for name, lat, lng in zip(df2.store, df2.lat, df2.lng):

    # 마커 찍기
    folium.Marker(
        [lat, lng],
        popup=name
    # 서울 맵에다가 넣는다.
    ).add_to(korea_map)


korea_map

In [8]:
korea_map.save("뚜레쥬르.html")